# Using ASE with our conformer search tool

Because the overall goal is to use ASE for the development of `RMG-Cat`, I decided I might as well try using ASE to get some stuff done

**The following methods and classes have been moved to the `multi_molecule.py` file in `python_code`**

In [1]:
import ase
from ase import Atom, Atoms

import os
import sys
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from rdkit.Chem.rdMolTransforms import *

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.kinetics import PDepArrhenius, PDepKineticsModel

from rmgpy.data.rmg import RMGDatabase
from rmgpy.data.kinetics import KineticsDepository, KineticsRules
from rmgpy.qm.main import QMCalculator, QMSettings
from rmgpy.qm.qmdata import QMData
from rmgpy.qm.reaction import QMReaction
from rmgpy.qm.molecule import QMMolecule

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def view_mol(mol):
    """
    A function designed to visulaize rdkit molecules
    
    Input: 
    * mol (an rdkit molecule object)
    
    Output: 
    * 3D figure of the molecule from py3Dmol
    
    """
    mb  = Chem.MolToMolBlock(mol)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb, "sdf")
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()

In [3]:
RDMol = AllChem.MolFromSmiles("CCCC")
RDMol = Chem.AddHs(RDMol)
AllChem.EmbedMolecule(RDMol)
view_mol(RDMol)

In [4]:
mol_list = AllChem.MolToMolBlock(RDMol).split('\n')
ase_atoms = []
for i, line in enumerate(mol_list):
    print i, line
    if i > 3:
        
        try:
            atom0, atom1, bond, rest = line
            atom0 = int(atom0)
            atom0 = int(atom1)
            bond = float(bond)
            #print atom0, atom1, bond
            #print
        except ValueError:
            try:
                x, y, z, symbol = line.split()[0:4]
                x = float(x)
                y = float(y)
                z = float(z)

                ase_atoms.append(Atom(symbol=symbol, position=(x,y,z)))
                #print x, y, z
                #print symbol
                #print
            except:
                continue
        
    
ase_mol = Atoms(ase_atoms)
ase_mol

0 
1      RDKit          3D
2 
3  14 13  0  0  0  0  0  0  0  0999 V2000
4    -1.7636    0.1460    0.0353 C   0  0  0  0  0  0  0  0  0  0  0  0
5    -0.5139   -0.3600   -0.6198 C   0  0  0  0  0  0  0  0  0  0  0  0
6     0.6532   -0.5235    0.3125 C   0  0  0  0  0  0  0  0  0  0  0  0
7     1.6867    0.5554    0.1870 C   0  0  0  0  0  0  0  0  0  0  0  0
8    -1.6009    0.7193    0.9532 H   0  0  0  0  0  0  0  0  0  0  0  0
9    -2.4883   -0.6707    0.2844 H   0  0  0  0  0  0  0  0  0  0  0  0
10    -2.3221    0.7991   -0.6824 H   0  0  0  0  0  0  0  0  0  0  0  0
11    -0.2921    0.2524   -1.5052 H   0  0  0  0  0  0  0  0  0  0  0  0
12    -0.7531   -1.3868   -0.9960 H   0  0  0  0  0  0  0  0  0  0  0  0
13     0.2632   -0.4839    1.3585 H   0  0  0  0  0  0  0  0  0  0  0  0
14     1.1343   -1.5318    0.2303 H   0  0  0  0  0  0  0  0  0  0  0  0
15     2.6866    0.1178    0.4147 H   0  0  0  0  0  0  0  0  0  0  0  0
16     1.5357    1.4087    0.8762 H   0  0  0  0  0  0  0

Atoms(symbols='C4H10', pbc=False)

In [5]:
ase_atoms

[Atom('C', [-1.7636000000000001, 0.14599999999999999, 0.035299999999999998]),
 Atom('C', [-0.51390000000000002, -0.35999999999999999, -0.61980000000000002]),
 Atom('C', [0.6532, -0.52349999999999997, 0.3125]),
 Atom('C', [1.6867000000000001, 0.5554, 0.187]),
 Atom('H', [-1.6009, 0.71930000000000005, 0.95320000000000005]),
 Atom('H', [-2.4883000000000002, -0.67069999999999996, 0.28439999999999999]),
 Atom('H', [-2.3220999999999998, 0.79910000000000003, -0.68240000000000001]),
 Atom('H', [-0.29210000000000003, 0.25240000000000001, -1.5052000000000001]),
 Atom('H', [-0.75309999999999999, -1.3868, -0.996]),
 Atom('H', [0.26319999999999999, -0.4839, 1.3585]),
 Atom('H', [1.1343000000000001, -1.5318000000000001, 0.2303]),
 Atom('H', [2.6865999999999999, 0.1178, 0.41470000000000001]),
 Atom('H', [1.5357000000000001, 1.4087000000000001, 0.87619999999999998]),
 Atom('H', [1.7743, 0.95799999999999996, -0.84860000000000002])]

# Classes to handle torsions and molecules from each type of molecule

In [6]:
import ase
from ase import Atom, Atoms
import rmgpy
from rmgpy.molecule import Molecule
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.rdMolTransforms import *
import py3Dmol
import numpy as np
from numpy import pi

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.rdMolTransforms import *
import ase
from ase import Atom, Atoms
import rmgpy
from rmgpy.molecule import Molecule
import py3Dmol
import numpy as np
from numpy import pi

class Torsion():
    """
    A class that acts as a container for torsion information.
    The information stored is as follows:

    * indicies (list of ints): indicies of atoms that describe the dihedral
    * dihedral (float): the angle (in degrees) of the dihedral
    * LHS (list of ints): indices of atoms that are branced off the left side of the torsion
    * RHS (list of ints): indices of atoms that are branced off the right side of the torsion
    """

    def __init__(self, indices, dihedral, LHS, RHS):
        self.indices = indices
        self.dihedral = dihedral
        self.LHS = LHS
        self.RHS = RHS




class Multi_Molecule():
    """
    A class that allows for one to create RMG, RDKit and ASE
    molecules from a single string with identical atom indicies

    Inputs:
    * smiles (str): a SMILES string that describes the molecule of interest
    """
    def __init__(self, smiles):

        self.smiles = smiles

        # Make these more explicit (make rmg mol from smiles)

        self.get_rmg_molecule()
        self.get_rdkit_molecule()
        self.set_rmg_coords("RDKit")
        self.get_ase_molecule()
        self.get_torsion_list()
        self.get_torsions()


    def get_rmg_molecule(self):
        """
        A method to obtain and set the RMG Molecule
        """
        self.rmg_molecule = Molecule(SMILES=self.smiles)
    def get_rdkit_molecule(self):
        """
        A method to create an RDKit Molecule from the rmg_molecule.
        Indicies will be the same as in the RMG Molecule
        """

        RDMol = self.rmg_molecule.toRDKitMol(removeHs=False)

        rdkit.Chem.AllChem.EmbedMolecule(RDMol)

        self.rdkit_molecule = RDMol

    def get_ase_molecule(self):
        """
        A method to create an ASE Molecule from the rdkit_molecule.
        Indicies will be the same as in the RMG and RDKit Molecule.
        """
        mol_list = AllChem.MolToMolBlock(self.rdkit_molecule).split('\n')
        ase_atoms = []
        for i, line in enumerate(mol_list):

            if i > 3:

                try:
                    atom0, atom1, bond, rest = line
                    atom0 = int(atom0)
                    atom0 = int(atom1)
                    bond = float(bond)

                except ValueError:
                    try:
                        x, y, z, symbol = line.split()[0:4]
                        x = float(x)
                        y = float(y)
                        z = float(z)
                        #print symbol

                        ase_atoms.append(Atom(symbol=symbol, position=(x,y,z)))

                    except:
                        continue

        self.ase_molecule = Atoms(ase_atoms)
        return self.ase_molecule

    def view_mol(self):
        """
        A method designed to create a 3D figure of the Multi_Molecule with py3Dmol
        """
        mb  = Chem.MolToMolBlock(self.rdkit_molecule)
        p = py3Dmol.view(width=400, height=400)
        p.addModel(mb, "sdf")
        p.setStyle({'stick':{}})
        p.setBackgroundColor('0xeeeeee')
        p.zoomTo()
        return p.show()


    def get_torsion_list(self):
        """
        A method to return a list of the possible torsions in a Multi_Molecule.
        This uses the RDKit framework to do this.
        """
        RDMol = self.rdkit_molecule
        torsion_list = []
        for bond1 in RDMol.GetBonds():
            atom1 = bond1.GetBeginAtom()
            atom2 = bond1.GetEndAtom()
            if atom1.IsInRing() or atom2.IsInRing():
                # Making sure that bond1 we're looking at are in a ring
                continue

            bond_list1 = list(atom1.GetBonds())
            bond_list2 = list(atom2.GetBonds())

            if not len(bond_list1) > 1 and not len(bond_list2) > 1:
                # Making sure that there are more than one bond attached to
                # the atoms we're looking at
                continue

            # Getting the 0th and 3rd atom and insuring that atoms
            # attached to the 1st and 2nd atom are not terminal hydrogens
            # We also make sure that all of the atoms are properly bound together

            # If the above are satisified, we append a tuple of the torsion our torsion_list
            got_atom0 = False
            got_atom3 = False

            for bond0 in bond_list1:
                atomX = bond0.GetOtherAtom(atom1)
                if atomX.GetAtomicNum() == 1 and len(atomX.GetBonds()) == 1:
                    # This means that we have a terminal hydrogen, skip this
                    # NOTE: for H_abstraction TSs, a non teminal H should exist
                    continue
                if atomX.GetIdx() != atom2.GetIdx():
                    got_atom0 = True
                    atom0 = atomX

            for bond2 in bond_list2:
                atomY = bond2.GetOtherAtom(atom2)
                if atomY.GetAtomicNum() == 1 and len(atomY.GetBonds()) == 1:
                    # This means that we have a terminal hydrogen, skip this
                    continue
                if atomY.GetIdx() != atom1.GetIdx():
                    got_atom3 = True
                    atom3 = atomY

            if not (got_atom0 and got_atom3):
                # Making sure atom0 and atom3 were not found
                continue

            # Looking to make sure that all of the atoms are properly bonded to eached
            if (
                RDMol.GetBondBetweenAtoms(atom0.GetIdx(), atom1.GetIdx()) and
                RDMol.GetBondBetweenAtoms(atom1.GetIdx(), atom2.GetIdx()) and
                RDMol.GetBondBetweenAtoms(atom2.GetIdx(), atom3.GetIdx())   ) :

                torsion_tup = (atom0.GetIdx(), atom1.GetIdx(), atom2.GetIdx(), atom3.GetIdx())
                torsion_list.append(torsion_tup)

        self.torsion_list = torsion_list
        return self.torsion_list

    def get_torsions(self):
        torsions = []
        for indices in self.torsion_list:
            i, j, k, l = indices

            dihedral = self.ase_molecule.get_dihedral(i,j,k,l)
            tor = Torsion(indices=indices, dihedral=dihedral, LHS=[], RHS=[])
            LHS = self.get_LHS(tor)
            RHS = self.get_RHS(tor)

            torsions.append(Torsion(indices, dihedral, LHS, RHS))
        self.torsions = torsions
        return self.torsions

    def get_LHS(self, Torsion):

        rdkit_atoms = self.rdkit_molecule.GetAtoms()

        L1, L0, R0, R1 = Torsion.indices
        rd_atom_L1 = rdkit_atoms[L1]
        rd_atom_L0 = rdkit_atoms[L0]
        rd_atom_R0 = rdkit_atoms[R0]
        rd_atom_R1 = rdkit_atoms[R1]

        # trying to get the left hand side of this torsion
        LHS_atoms_index = [rd_atom_L0.GetIdx(), rd_atom_L1.GetIdx()]
        RHS_atoms_index = [rd_atom_R0.GetIdx(), rd_atom_R1.GetIdx()]

        complete_LHS = False
        i = 0
        atom_index = LHS_atoms_index[0]
        while complete_LHS == False:
            try:
                LHS_atom = rdkit_atoms[atom_index]
                for neighbor in LHS_atom.GetNeighbors():
                    if (neighbor.GetIdx() in LHS_atoms_index) or (neighbor.GetIdx() in RHS_atoms_index):
                        continue
                    else:
                        LHS_atoms_index.append(neighbor.GetIdx())
                i +=1
                atom_index = LHS_atoms_index[i]

            except IndexError:
                complete_LHS = True

        return LHS_atoms_index

    def get_RHS(self, Torsion):

        rdkit_atoms = self.rdkit_molecule.GetAtoms()

        L1, L0, R0, R1 = Torsion.indices
        rd_atom_L1 = rdkit_atoms[L1]
        rd_atom_L0 = rdkit_atoms[L0]
        rd_atom_R0 = rdkit_atoms[R0]
        rd_atom_R1 = rdkit_atoms[R1]

        # trying to get the left hand side of this torsion
        LHS_atoms_index = [rd_atom_L0.GetIdx(), rd_atom_L1.GetIdx()]
        RHS_atoms_index = [rd_atom_R0.GetIdx(), rd_atom_R1.GetIdx()]

        complete_RHS = False
        i = 0
        atom_index = RHS_atoms_index[0]
        while complete_RHS == False:
            try:
                RHS_atom = rdkit_atoms[atom_index]
                for neighbor in RHS_atom.GetNeighbors():
                    if (neighbor.GetIdx() in RHS_atoms_index) or (neighbor.GetIdx() in LHS_atoms_index):
                        continue
                    else:
                        RHS_atoms_index.append(neighbor.GetIdx())
                i +=1
                atom_index = RHS_atoms_index[i]

            except IndexError:
                complete_RHS = True

        return RHS_atoms_index

    def set_rmg_coords(self, molecule_base):

        if molecule_base == "RDKit":
            mol_list = AllChem.MolToMolBlock(self.rdkit_molecule).split('\n')
            for i, atom in enumerate(self.rmg_molecule.atoms):

                j = i + 4
                coords = mol_list[j].split()[:3]

                for k, coord in enumerate(coords):

                    coords[k] = float(coord)
                atom.coords = np.array(coords)

        elif molecule_base == "ASE":
            for i, position in enumerate(self.ase_molecule.get_positions()):
                self.rmg_molecule.atoms[i].coords = position




    def update_geometry_from_rdkit_mol(self):
        # In order to update the ase molecule you simply need to rerun the get_ase_molecule method
        self.get_ase_molecule()
        self.set_rmg_coords("RDKit")



    def update_geometry_from_ase_mol(self):
        #TODO: set the positions of the rdkit molecule from ase

        self.set_rmg_coords("ASE")





   


In [7]:
mol = Multi_Molecule("CCCC")


### Looking at BoundsMatricies from rdkit and ase

In [8]:
a = rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol.rdkit_molecule)
b = mol.ase_molecule.get_all_distances()

In [9]:
mol.view_mol()

In [10]:
b

array([[ 0.        ,  1.50066868,  1.51253774,  2.44383883,  1.11877961,
         1.11920973,  2.19357277,  2.16667724,  2.15129021,  2.13101067,
         2.19411845,  2.53378938,  2.69958142,  3.40209429],
       [ 1.50066868,  0.        ,  2.5167035 ,  1.52471139,  2.19752345,
         2.1442347 ,  1.10241541,  1.12028073,  3.437615  ,  2.98676609,
         2.6754082 ,  2.13404662,  2.11189794,  2.15799118],
       [ 1.51253774,  2.5167035 ,  0.        ,  3.58134189,  2.15140926,
         2.12947603,  2.57512221,  3.29426844,  1.12016772,  1.11569517,
         1.09967254,  3.89364255,  3.4873469 ,  4.5016116 ],
       [ 2.44383883,  1.52471139,  3.58134189,  0.        ,  2.44448229,
         3.17502488,  2.19192347,  2.101343  ,  4.27011628,  4.3130593 ,
         3.57593357,  1.10729427,  1.09671635,  1.10994478],
       [ 1.11877961,  2.19752345,  2.15140926,  2.44448229,  0.        ,
         1.77514452,  3.00147331,  2.90390716,  2.30393032,  2.99242701,
         2.73090871,  2.18

In [11]:
# Looking at the differences between the distance matricies from rdkit and ase

abs(a-b)

array([[  0.00000000e+00,   2.33313224e-02,   1.14622616e-02,
          6.89518025e-02,   6.21182979e-04,   1.91060288e-04,
          1.49382582e-03,   2.83893524e-02,   4.37763836e-02,
          6.40559278e-02,   9.48147033e-04,   9.05116717e-01,
          7.39324679e-01,   3.68118057e-02],
       [  3.33132238e-03,   0.00000000e+00,   3.91287556e-03,
          7.11385804e-04,   2.45685282e-03,   5.08318985e-02,
          1.69853877e-02,   8.79933333e-04,   1.29109429e-03,
          4.52140001e-01,   7.63497894e-01,   6.10199756e-02,
          8.31686563e-02,   3.70754102e-02],
       [  8.53773837e-03,   8.39128756e-02,   0.00000000e+00,
          2.29377594e-01,   4.36573374e-02,   6.55905600e-02,
          8.63783888e-01,   1.44637651e-01,   7.66924707e-04,
          3.70562594e-03,   1.97282573e-02,   9.96106357e+02,
          9.96512653e+02,   9.95498388e+02],
       [  1.10481975e-02,   2.07113858e-02,   1.05657472e+00,
          0.00000000e+00,   9.94423808e-01,   2.63881210e-0

## Finding the LHS and RHS of a torsion (for use of ASE)

In [12]:
mol = Multi_Molecule("NOCC")
i,j,k,l = mol.torsions[0].indices
molecule = mol.rdkit_molecule
tup = molecule.GetConformer(0)
GetDihedralDeg(tup, i,j,k,l)

157.30480480568735

In [13]:
print mol.torsion_list
rd_atoms = molecule.GetAtoms()

L1, L0, R0, R1 = mol.torsion_list[0]
rd_atom_L1 = rd_atoms[L1]
rd_atom_L0 = rd_atoms[L0]
rd_atom_R0 = rd_atoms[R0]
rd_atom_R1 = rd_atoms[R1]

# trying to get the left hand side of this torsion
LHS_atoms_index = [rd_atom_L0.GetIdx(), rd_atom_L1.GetIdx()]
RHS_atoms_index = [rd_atom_R0.GetIdx(), rd_atom_R1.GetIdx()]

complete_LHS = False
i = 0
atom_index = LHS_atoms_index[0]
while complete_LHS == False:
    try:
        LHS_atom = rd_atoms[atom_index]
        for neighbor in LHS_atom.GetNeighbors():
            if (neighbor.GetIdx() in LHS_atoms_index) or (neighbor.GetIdx() in RHS_atoms_index):
                continue
            else:
                LHS_atoms_index.append(neighbor.GetIdx())
        i +=1
        atom_index = LHS_atoms_index[i]
   
    except IndexError:
        complete_LHS = True

        

print LHS_atoms_index
view_mol(molecule)

[(1, 0, 3, 2)]
[0, 1, 4, 5, 8, 6, 7]


## Writing formal functions / methods for the above

In [14]:
def get_LHS(self, Torsion):
    
    rdkit_atoms = self.rdkit_molecule.GetAtoms()

    L1, L0, R0, R1 = Torsion.indicies
    rd_atom_L1 = rdkit_atoms[L1]
    rd_atom_L0 = rdkit_atoms[L0]
    rd_atom_R0 = rdkit_atoms[R0]
    rd_atom_R1 = rdkit_atoms[R1]

    # trying to get the left hand side of this torsion
    LHS_atoms_index = [rd_atom_L0.GetIdx(), rd_atom_L1.GetIdx()]
    RHS_atoms_index = [rd_atom_R0.GetIdx(), rd_atom_R1.GetIdx()]

    complete_LHS = False
    i = 0
    atom_index = LHS_atoms_index[0]
    while complete_LHS == False:
        try:
            LHS_atom = rdkit_atoms[atom_index]
            for neighbor in LHS_atom.GetNeighbors():
                if (neighbor.GetIdx() in LHS_atoms_index) or (neighbor.GetIdx() in RHS_atoms_index):
                    continue
                else:
                    LHS_atoms_index.append(neighbor.GetIdx())
            i +=1
            atom_index = LHS_atoms_index[i]

        except IndexError:
            complete_LHS = True

    return LHS_atoms_index
    
def get_RHS(self, Torsion):
    
    rdkit_atoms = self.rdkit_molecule.GetAtoms()

    L1, L0, R0, R1 = Torsion.indicies
    rd_atom_L1 = rdkit_atoms[L1]
    rd_atom_L0 = rdkit_atoms[L0]
    rd_atom_R0 = rdkit_atoms[R0]
    rd_atom_R1 = rdkit_atoms[R1]

    # trying to get the left hand side of this torsion
    LHS_atoms_index = [rd_atom_L0.GetIdx(), rd_atom_L1.GetIdx()]
    RHS_atoms_index = [rd_atom_R0.GetIdx(), rd_atom_R1.GetIdx()]

    complete_RHS = False
    i = 0
    atom_index = RHS_atoms_index[0]
    while complete_RHS == False:
        try:
            RHS_atom = rdkit_atoms[atom_index]
            for neighbor in RHS_atom.GetNeighbors():
                if (neighbor.GetIdx() in RHS_atoms_index) or (neighbor.GetIdx() in LHS_atoms_index):
                    continue
                else:
                    LHS_atoms_index.append(neighbor.GetIdx())
            i +=1
            atom_index = RHS_atoms_index[i]

        except IndexError:
            complete_RHS = True

    return RHS_atoms_index